### Dự đoán điểm số mà người dùng có thể chấm cho một phim và xếp hạng các phim để recommend cho người dùng dựa vào dữ liệu implicit (không rõ điểm số)

- Ta sử dụng phương pháp ALS, để dự đoán điểm số của người dùng dành cho các phim
- Từ đó xây dựng lại dataset với dự đoán điểm số của người dùng vào các ô trống (Điểm -1)

In [1]:
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import metrics_eval
import matplotlib.pyplot as plt



c:\Users\LU QUOC PHAP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
anime = pd.read_csv("Dataset/preprocessed_anime.csv")
rating = pd.read_csv("Dataset/rating.csv")
train_rating = pd.read_csv("Dataset/train_rating.csv")
test_rating = pd.read_csv("Dataset/test_rating.csv")

- Đọc dữ liệu anime và rating

In [3]:
anime.head(10)

,anime_id,name,genre,type,episodes,rating,members,n_genres,genre_primary
0,32281,Kimi no Na wa.,"Drama,Romance,School,Supernatural",Movie,1,9.37,200630,4,drama
1,5114,Fullmetal Alchemist: Brotherhood,"Action,Adventure,Drama,Fantasy,Magic,Military,...",TV,64,9.26,793665,7,action
2,28977,Gintama°,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.25,114262,7,action
3,9253,Steins;Gate,"Sci-Fi,Thriller",TV,24,9.17,673572,2,sci-fi
4,9969,Gintama&#039;,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.16,151266,7,action
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy,Drama,School,Shounen,Sports",TV,10,9.15,93351,5,comedy
6,11061,Hunter x Hunter (2011),"Action,Adventure,Shounen,Super Power",TV,148,9.13,425855,4,action
7,820,Ginga Eiyuu Densetsu,"Drama,Military,Sci-Fi,Space",OVA,110,9.11,80679,4,drama
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",Movie,1,9.10,72534,7,action
9,15417,Gintama&#039;: Enchousen,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,13,9.11,81109,7,action


In [4]:
rating.head(10)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
5,1,355,-1
6,1,356,-1
7,1,442,-1
8,1,487,-1
9,1,846,-1


In [5]:
print(f"rating shape: {rating.shape}")
print(f"train_rating shape: {train_rating.shape}")
print(f"test_rating shape: {test_rating.shape}")

rating shape: (7813755, 3)
train_rating shape: (6260024, 4)
test_rating shape: (1553731, 4)


### Trước khi đưa dữ liệu và ma trận ALS
- Ta cần xử lý lại điểm số của một số tương tác dương
- Vì ALS sẽ đánh giá độ yêu thích của người dùng với các phim càng thích thì có giá trị trên ma trận càng cao.
- Tuy nhiên, giá trị -1 (đã xem) lại có giá trị thấp nhất. Vì thế trước khi scale về thang [0, 1] ta cho những giá trị -1 ánh xạ thành 5 (trung tính)

In [6]:
def confidence_mapping(r):
    if r == -1: return 5
    elif r >= 8: return 10
    elif r >= 5: return 4
    else: return 1
rating['confidence'] = rating['rating'].apply(confidence_mapping)
train_rating['confidence'] = train_rating['rating'].apply(confidence_mapping)
test_rating['confidence'] = test_rating['rating'].apply(confidence_mapping)


In [7]:
# Tạo danh sách đầy đủ user_id và item_id từ rating gốc
all_users = sorted(rating["user_id"].unique())
all_items = sorted(rating["anime_id"].unique())

user_map = {u: i for i, u in enumerate(all_users)}
item_map = {it: j for j, it in enumerate(all_items)}

# Gán lại chỉ số index cho cả train và test
train_rating["user_idx"] = train_rating["user_id"].map(user_map)
train_rating["item_idx"] = train_rating["anime_id"].map(item_map)
test_rating["user_idx"]  = test_rating["user_id"].map(user_map)
test_rating["item_idx"]  = test_rating["anime_id"].map(item_map)


user_items = csr_matrix(
    (train_rating["confidence"].values, (train_rating["user_idx"].values, train_rating["item_idx"].values)),
    shape=(len(all_users), len(all_items)),
    dtype=np.float32
)


In [8]:
user_items_test = csr_matrix(
    (test_rating["confidence"].values, (test_rating["user_idx"].values, test_rating["item_idx"].values)),
    shape=(len(all_users), len(all_items)),
    dtype=np.float32
)

In [9]:
user_items_test.shape

(73518, 11202)

In [10]:

# Huấn luyện mô hình ALS
model = AlternatingLeastSquares(factors=150, regularization=0.1, iterations=100)
model.fit(user_items)



c:\Users\LU QUOC PHAP\AppData\Local\Programs\Python\Python312\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 100/100 [04:17<00:00,  2.57s/it]


In [11]:

userids = test_rating['user_idx']
ids, scores = model.recommend(userids, user_items[userids], 20)

In [12]:
reverse_item_map = {v: k for k, v in item_map.items()}
reverse_user_map = {v: k for k, v in user_map.items()}

In [13]:
predicted = {int(reverse_user_map.get(u)): [int(reverse_item_map.get(i)) for i in row] for u, row in zip(userids, ids)}

ground_truth = (
    test_rating
    .groupby('user_id')['anime_id']  
    .apply(set)
    .to_dict()
)

In [15]:
result = metrics_eval.evaluate_all(predicted=predicted, ground_truth=ground_truth, k=15)

In [16]:
print(result)

{'Precision@15': 0.2980784286149591, 'Recall@15': 0.31338123987133054, 'MAP@15': 0.2831827437612844}


- Đã chạy lại (5)